In [7]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer


from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
import pickle
from sklearn.model_selection import KFold, StratifiedKFold

# Load Data

In [8]:

train_doc = pd.read_pickle('../../data/fe_doc/train_doc.pkl')
test_doc = pd.read_pickle('../../data/fe_doc/test_doc.pkl')



holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_index = pickle.load(open(FILE.train_index.value,'rb'))
train_cv = train_doc.loc[train_index].copy()
holdout = train_doc.loc[holdout_index].copy()

holdout_label = holdout.click.values
label_name = FILE.Vectorize_label_format.value.format('holdout')
# pickle.dump(holdout.click.values,open(label_name,'wb'))


In [9]:
ct = CountVectorizer(token_pattern=r'(?u)\b[\w.-]+\b',
                max_features=500000)
ctvector = ct.fit(pd.concat([train_doc['text'],test_doc['text']]))

# train_cv_matrix = ctvector.transform(train_cv['text'])
holdout_matrix = ctvector.transform(holdout['text'])
test_matrix = ctvector.transform(test_doc['text'])

In [10]:
test_matrix.shape

(40024, 25422)

In [12]:
s1 = np.sum(test_matrix,axis=0)
s1.shape

(1, 25422)

In [19]:
np.log2(s1+1)

matrix([[ 8.37937837,  1.        ,  5.357552  , ...,  7.83920379,
         13.69185255,  5.169925  ]])

In [14]:
s2 = test_matrix.multiply(s1)

In [16]:
np.sum(s2,axis=0)

matrix([[   110224,         1,      1600, ...,     51984, 175085824,
              1225]], dtype=int64)

In [5]:
# # n_fold = 5
# # sf = StratifiedKFold(n_splits=5,shuffle= True,random_state=20)
# # train_index_list = []
# # val_index_list = []
# # fold = 0
# # for t,v in sf.split(train_cv,train_cv.click):
# #     print('here')
# #     train_cur_index = train_cv.iloc[t].index
# #     train_cur_label = train_cv.iloc[t].click.values
# #     index_name = FILE.Vectorize_index_format.value.format('train_fold_{}'.format(fold))
# #     label_name = FILE.Vectorize_label_format.value.format('train_fold_{}'.format(fold))
# #     pickle.dump(train_cur_index,open(index_name,'wb'))
# #     pickle.dump(train_cur_label,open(label_name,'wb'))
# #     val_cur_index = train_cv.iloc[v].index
# #     val_cur_label =  train_cv.iloc[v].click.values
# #     index_name = FILE.Vectorize_index_format.value.format('val_fold_{}'.format(fold))
# #     label_name = FILE.Vectorize_label_format.value.format('val_fold_{}'.format(fold))
# #     pickle.dump(val_cur_index,open(index_name,'wb'))
# #     pickle.dump(val_cur_label,open(label_name,'wb'))
# #     fold += 1
# label_name = FILE.Vectorize_label_format.value.format('holdout')
# pickle.dump(holdout.click.values,open(label_name,'wb'))

In [4]:
day_index = pickle.load(open(FILE.train_7_fold_index.value,'rb'))

for key,(v,t) in day_index.items():
    index_name = FILE.Vectorize_index_format.value.format('train_fold_{}'.format(key))
    print('train index length:{}'.format(len(t)))
    t_labels = train_cv.loc[t,'click'].values
    label_name = FILE.Vectorize_label_format.value.format('train_fold_{}'.format(key))
    print('train label length:{}'.format(len(t_labels)))
    matrix_train = ctvector.transform(train_cv.loc[t]['text'])
    print('train matrix shape:{}'.format(matrix_train.shape))
    name = 'train_fold_{}'.format(key)
    save_name = FILE.tfidfVectorize_format.value.format(name)
    pickle.dump(matrix_train,open(save_name,'wb'))
    pickle.dump(t,open(index_name,'wb'))
    pickle.dump(t_labels,open(label_name,'wb'))
    
    #### validation
    index_name = FILE.Vectorize_index_format.value.format('val_fold_{}'.format(key))
    print('val index length:{}'.format(len(v)))
    v_labels = train_cv.loc[v,'click'].values
    label_name = FILE.Vectorize_label_format.value.format('val_fold_{}'.format(key))
    print('val label length:{}'.format(len(v_labels)))
    matrix_val = ctvector.transform(train_cv.loc[v]['text'])
    print('val matrix shape:{}'.format(matrix_val.shape))
    name = 'val_fold_{}'.format(key)
    save_name = FILE.tfidfVectorize_format.value.format(name)
    pickle.dump(matrix_val,open(save_name,'wb'))
    pickle.dump(v,open(index_name,'wb'))
    pickle.dump(v_labels,open(label_name,'wb'))
    
    
save_name = FILE.tfidfVectorize_format.value.format('holdout')
pickle.dump(holdout_matrix,open(save_name,'wb'))
print(holdout_matrix.shape)

save_name = FILE.tfidfVectorize_format.value.format('test')
pickle.dump(test_matrix,open(save_name,'wb'))
print(test_matrix.shape)

train index length:826135
train label length:826135
train matrix shape:(826135, 25422)
val index length:135491
val label length:135491
val matrix shape:(135491, 25422)
train index length:818892
train label length:818892
train matrix shape:(818892, 25422)
val index length:142734
val label length:142734
val matrix shape:(142734, 25422)
train index length:816463
train label length:816463
train matrix shape:(816463, 25422)
val index length:145163
val label length:145163
val matrix shape:(145163, 25422)
train index length:832563
train label length:832563
train matrix shape:(832563, 25422)
val index length:129063
val label length:129063
val matrix shape:(129063, 25422)
train index length:820939
train label length:820939
train matrix shape:(820939, 25422)
val index length:140687
val label length:140687
val matrix shape:(140687, 25422)
train index length:810325
train label length:810325
train matrix shape:(810325, 25422)
val index length:151301
val label length:151301
val matrix shape:(151301,

In [5]:
day_index = pickle.load(open(FILE.train_7_fold_index.value,'rb'))
val_index_list = []

for key,(v,t) in day_index.items():

    val_index_list.extend(list(v))
val_index_list.extend(list(holdout_index))


In [6]:
len(set(val_index_list))

1001650

In [8]:
# n_fold = 5
# sf = StratifiedKFold(n_splits=5,shuffle= True,random_state=20)
# train_index_list = []
# val_index_list = []
# for t,v in sf.split(train_cv,train_cv.click):
#     train_index_list.append(t)
#     val_index_list.append(v)
    
# for fold in range(len(train_index_list)):
#     matrix_train = ctvector.transform(train_cv.iloc[train_index_list[fold]]['text'])
#     print(matrix_train.shape)
#     name = 'train_fold_{}'.format(fold)
#     save_name = FILE.tfidfVectorize_format.value.format(name)
#     pickle.dump(matrix_train,open(save_name,'wb'))
#     matrix_val = ctvector.transform(train_cv.iloc[val_index_list[fold]]['text'])
#     print(matrix_val.shape)
#     name = 'val_fold_{}'.format(fold)
#     save_name = FILE.tfidfVectorize_format.value.format(name)
#     pickle.dump(matrix_val,open(save_name,'wb'))
    
# save_name = FILE.tfidfVectorize_format.value.format('holdout')
# pickle.dump(holdout_matrix,open(save_name,'wb'))
# print(holdout_matrix.shape)

# save_name = FILE.tfidfVectorize_format.value.format('test')
# pickle.dump(test_matrix,open(save_name,'wb'))
# print(test_matrix.shape)

In [ ]:
for fold in range(len(train_index_list)):

# Define NN model

In [4]:

def get_nn_model(param):
    layer1 = param.get('layer1')
    layer2 = param.get('layer2')
    layer3 = param.get('layer3')
    input_shape = param.get('input_shape')
    
    sequence_input = Input(shape=(input_shape, ))
    x = Dense(layer1, activation='relu')(sequence_input)
    x = Dropout(0.5)(x)
    
    if layer2 is not None:
        x = Dense(layer2, activation='relu')(x)
        x = Dropout(0.5)(x)
    
    if layer2 is not None and layer3 is not None:
        x = Dense(layer3, activation='relu')(x)
        x = Dropout(0.2)(x)
        
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model

def train_each_epoch(x,y,batch_size,model):
    x,y = shuffle(x,y)
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              )
    return model

In [11]:
param_list = []
for layer1 in [512,1024,2048]:
    for layer2 in [None,32,512,1024]:
        for layer3 in [None,16,32,128]:
            if layer2 is None and layer3 is not None:
                continue
            param = {'input_shape':train_cv_matrix.shape[1],
                     'layer1':layer1,
                     'layer2':layer2,
                     'layer3':layer3}
            param_list.append(param)
print(len(param_list))

def get_file_name(param,vectorize='countV'):
    layer1 = param.get('layer1')
    layer2 = param.get('layer2')
    layer3 = param.get('layer3')
    return '{}_layer1_{}_layer2_{}_layer3_{}.pkl'.format(vectorize,layer1,layer2,layer3)

def save_report(params,report_path):
    """
    params. dataframe report params.
    """
    try:
        saved_report = pd.read_csv(report_path)
    except:
        saved_report = pd.DataFrame()
    new_report = pd.DataFrame({'fileName':params['file_Name'],
                               'cv_mean':params['cv_mean'],
                               'holdout':params['holdout']})
    saved_report = pd.concat([saved_report,new_report]).reset_index(drop=True)
    saved_report.to_csv(report_path)
    print('saved report to path {}'.format(report_path))
    
def save_preds(param,preds_params,path='../../data/nn/{}/{}'):
    """
    param: nn layer info.
    preds_params:key should be train/test/holdout and values should be coresponding predict dataframe
    """
    file_name = get_file_name(param)
    for key in preds_params:
        saving_path = path.format(key,file_name)
        preds_params[key].to_pickle(saving_path)
        print('saving preds {} done!'.format(key))
        
def train_each_fold(param,fold,mode='countVectorize'):
    if mode != 'tfidf':
        train_name = FILE.countVectorize_format.value.format('train_fold_{}'.format(fold))
        val_name = FILE.countVectorize_format.value.format('val_fold_{}'.format(fold))
        holdout_name = FILE.countVectorize_format.value.format('holdout')
        test_name = FILE.countVectorize_format.value.format('test')
        
    else:
        train_name = FILE.tfidfVectorize_format.value.format('train_fold_{}'.format(fold))
        val_name = FILE.tfidfVectorize_format.value.format('val_fold_{}'.format(fold))
        holdout_name = FILE.tfidfVectorize_format.value.format('holdout')
        test_name = FILE.tfidfVectorize_format.value.format('test')
        
    y_train_name = FILE.Vectorize_label_format.value.format('train_fold_{}'.format(fold))
    y_val_name = FILE.Vectorize_label_format.value.format('val_fold_{}'.format(fold))
    y_holdout_name = FILE.Vectorize_label_format.value.format('holdout')
        
    x_train = pickle.load(open(train_name,'rb'))
    x_val = pickle.load(open(val_name,'rb'))
    x_holdout = pickle.load(open(holdout_name,'rb'))
    x_test = pickle.load(open(test_name,'rb'))
    y_train =  

39


# One fold Demo

In [78]:
model = get_nn_model(train_cv_matrix.shape[1])

In [79]:
model = train_each_epoch(train_cv_matrix,train_cv.click.values,2048,model)

Epoch 1/1
961626/961626 [==============================] - 203s 211us/step - loss: 0.4238 - acc: 0.8043


In [80]:
y_pred = model.predict(holdout_matrix,batch_size=3000,verbose=1)
score = log_loss(holdout.click.values,y_pred)
print(score)

40024/40024 [==============================] - 6s 156us/step
0.43059639321782484


In [81]:
model = train_each_epoch(train_cv_matrix,train_cv.click.values,2048,model)

Epoch 1/1
961626/961626 [==============================] - 197s 205us/step - loss: 0.4160 - acc: 0.8067


In [82]:
y_pred = model.predict(holdout_matrix,batch_size=6000,verbose=1)
score = log_loss(holdout.click.values,y_pred)
print(score)

40024/40024 [==============================] - 6s 157us/step
0.4276796885171615


In [66]:
model = train_each_epoch(train_cv_matrix,train_cv.click.values,2048,model)

Epoch 1/1
217088/961626 [=====>........................] - ETA: 1:45 - loss: 0.4105 - acc: 0.8074

KeyboardInterrupt: 

In [57]:
y_pred = model.predict(holdout_matrix,batch_size=6000,verbose=1)
score = log_loss(holdout.click.values,y_pred)
print(score)

40024/40024 [==============================] - 5s 125us/step
0.4267815168378254


In [58]:
model = train_each_epoch(train_cv_matrix,train_cv.click.values,2048,model)

Epoch 1/1
961626/961626 [==============================] - 137s 143us/step - loss: 0.4108 - acc: 0.8079


In [59]:
y_pred = model.predict(holdout_matrix,batch_size=6000,verbose=1)
score = log_loss(holdout.click.values,y_pred)
print(score)

40024/40024 [==============================] - 5s 128us/step
0.42833751105049117
